In [1]:
from __future__ import division
def feature_eng(y, fs, vis=False):
    
    import librosa
    import librosa.display
    import numpy as np
    import maad
    from soundsig.sound import BioSound
    import matplotlib.pyplot as plt
    import pandas as pd
    import warnings
    warnings.simplefilter('ignore')
    
    # spectrum
    ps = np.abs(np.fft.fft(y))**2
    time_step = 1/fs
    freqs = np.fft.fftfreq(y.size, time_step)
    ps = ps[0:int((len(ps)/2)-1)] # take out the negative freq
    freqs = freqs[0:int((len(freqs)/2)-1)]

    # melspectrogram
    S = librosa.feature.melspectrogram(y=y, sr=fs)
    S_dB = librosa.power_to_db(S, ref=np.max)
    
    # alpha indices
    Sxx_power,tn,fn,ext = maad.sound.spectrogram (y, fs, mode='psd')
    df_temporal_indices = maad.features.all_temporal_alpha_indices(y, fs)
    df_spectral_indices, _ = maad.features.all_spectral_alpha_indices(Sxx_power,tn,fn, extent=ext)
    df_indices = pd.concat([df_temporal_indices,df_spectral_indices], axis=1)
    
    # soundsig for modulation power spectrum
    
    myBioSound = BioSound(soundWave=y, fs=fs)
    myBioSound.mpsCalc(window=1, Norm = True)
    
    # reduce the dimension of MPS to one quadrant
    len1 = int(len(myBioSound.wf-1)/2)
    len2 = int(len(myBioSound.wt-1)/2)
    quad1 = myBioSound.mps[len1:,len2:]
    quad2 = np.fliplr(myBioSound.mps[len1:,:len2+1])
    mps = (quad1+quad2)/2
    wf = myBioSound.wf[len1:]
    wt = myBioSound.wt[len2:]
    
    if vis==True:
        plt.figure()
        plt.plot(np.arange(0,y.size/fs,1/fs), y)
        plt.show()
        
        fig, ax = plt.subplots()
        img = librosa.display.specshow(S_dB, x_axis='time',
                                 y_axis='mel', sr=fs,
                                 fmax=fs/2, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set(title='Mel-frequency spectrogram')
        
        DBNOISE=100
        plt.figure()
        plt.clf()
        cmap = plt.get_cmap('jet')
        ex = (myBioSound.wt.min(), myBioSound.wt.max(), myBioSound.wf.min()*1e3, myBioSound.wf.max()*1e3)
        logMPS = 10.0*np.log10(myBioSound.mps)
        maxMPS = logMPS.max()
        minMPS = maxMPS-DBNOISE
        logMPS[logMPS < minMPS] = minMPS
        plt.imshow(logMPS, interpolation='nearest', aspect='auto', origin='lower', cmap=cmap, extent=ex)
        plt.ylabel('Spectral Frequency (Cycles/KHz)')
        plt.xlabel('Temporal Frequency (Hz)')
        plt.colorbar()
        plt.ylim((0,myBioSound.wf.max()*1e3))
        plt.title('Modulation Power Spectrum')
        plt.show()
        
    
    return ps, freqs, S_dB, df_indices, mps, wt, wf
    

In [2]:
def preproc(file, fs=16000, vis=False):
    import librosa
    import numpy as np
    import matplotlib.pyplot as plt
    import noisereduce as nr
    
    raw_y, fs = librosa.load(file, sr=fs, duration=10, mono = True)
    y_mono_rs = raw_y - np.mean(raw_y) # remove DC
    rms = np.sqrt(np.mean(y_mono_rs**2)) # get rms
    y = y_mono_rs/(rms/0.1) # normalize the rms to 0.1
    
    fg_y = nr.reduce_noise(y=y, sr=fs)
    bg_y = y - fg_y
    
    
    if vis == True: print('++++++++++++++++++++++++ raw ++++++++++++++++++++++++')
    ps, freqs, S_dB, df_indices, mps, wt, wf = feature_eng(y, fs, vis)
    if vis == True: print('++++++++++++++++++++ foreground ++++++++++++++++++++')
    ps_fg, freqs_fg, S_dB_fg, df_indices_fg, mps_fg, wt_fg, wf_fg = feature_eng(fg_y, fs, vis)
    if vis == True: print('++++++++++++++++++++ background ++++++++++++++++++++')
    ps_bg, freqs_fg, S_dB_bg, df_indices_bg, mps_bg, wt_bg, wf_bg = feature_eng(bg_y, fs, vis)
    
    output = {'fs': fs, 'y': y, 'fg_y': fg_y, 'bg_y': bg_y,
             'df_indices': df_indices, 'mps': mps, 'wt': wt, 'wf': wf, 
             'df_indices_fg': df_indices_fg, 'mps_fg': mps_fg,
             'df_indices_bg': df_indices_bg, 'mps_bg': mps_bg}
    
    return output
    

In [3]:
# scan data directories
import glob
import pandas as pd

nature_file_list = []
nature_file_list += glob.glob('../data/raw/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/raw/youtube/NatureSoundscapes/*.m4a')
nature_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/raw/S2L_LULC/urban_0_25/*')

remove_list_nature = glob.glob('../data/raw/youtube/NatureSoundscapes/xRrqcK46roE_*.m4a')
nature_file_list = [ele for ele in nature_file_list if ele not in remove_list_nature]


city_file_list = []
city_file_list += glob.glob('../data/raw/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/raw/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/raw/S2L_LULC/urban_26_100/*')

remove_list_city += glob.glob('../data/raw/youtube/NomadicAmbience_city/F5t-G50wnCo_*.m4a')
city_file_list = [ele for ele in city_file_list if ele not in remove_list_city]


In [4]:
# as there are too many files in SONYC, so only a few are sampled
from random import seed, sample
len_diff = len(nature_file_list)-len(city_file_list) # how many SONYC files need to add to make the dataset balanced

seed(23)
SONYC_file_list = sample(glob.glob('../data/raw/SONYC/**/*.wav', recursive=True),len_diff)
city_file_list += SONYC_file_list

pd.DataFrame({'nature_file_list': nature_file_list}).to_csv('../data/raw/nature_file_list.csv')
pd.DataFrame({'city_file_list': city_file_list}).to_csv('../data/raw/city_file_list.csv')

In [5]:
print('There are '+str(len(nature_file_list))+' files per category.')

There are 2959 files per category.


In [6]:
# testing cell
# preproc(city_file_list[0])

In [7]:
import time
import pickle
import os

from joblib import Parallel, delayed

def run_preproc(file_name):
    save_file_name = '../data/interim/'+file_name[12:-4]+'.pkl'
    if not os.path.isfile(save_file_name): # run the script only if the file does not exist
        if not os.path.exists(save_file_name.rsplit('/', 1)[0]): # create the folder if the folder does not exist
            os.makedirs(save_file_name.rsplit('/', 1)[0]) # extract the folder of the file path
            

        start_time = time.time()

        output = preproc(file_name, vis = False)
        f = open(save_file_name,'wb') # create a binary pickle file 
        pickle.dump(output,f)
        f.close()

        print("--- %s seconds ---" % (time.time() - start_time))
        

# run the process in parallel
Parallel(n_jobs=-1)(delayed(run_preproc)(file_name) for file_name in nature_file_list+city_file_list)

--- 14.796768188476562 seconds ---


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


--- 14.747337818145752 seconds ---
--- 10.056734323501587 seconds ---


/Users/andrewchang/opt/anaconda3/envs/AcousticEnv/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


--- 14.710429906845093 seconds ---
--- 9.952238082885742 seconds ---
--- 10.250509023666382 seconds ---
--- 12.34776496887207 seconds ---
--- 12.648195028305054 seconds ---
--- 7.2346038818359375 seconds ---
--- 7.91603684425354 seconds ---
--- 8.133424997329712 seconds ---
--- 8.714755058288574 seconds ---
--- 8.771554708480835 seconds ---
--- 9.191856145858765 seconds ---
--- 9.608963966369629 seconds ---
--- 8.242719888687134 seconds ---
--- 8.031042098999023 seconds ---
--- 8.611571073532104 seconds ---
--- 8.836889266967773 seconds ---
--- 8.464759111404419 seconds ---
--- 8.908244848251343 seconds ---
--- 9.518594026565552 seconds ---
--- 8.159053087234497 seconds ---
--- 10.03777003288269 seconds ---
--- 11.291344165802002 seconds ---
--- 14.804182052612305 seconds ---
--- 9.965203285217285 seconds ---
--- 10.14972710609436 seconds ---
--- 12.59656810760498 seconds ---
--- 12.499889850616455 seconds ---
--- 7.329277038574219 seconds ---
--- 8.030877828598022 seconds ---
--- 8.26

--- 14.695176124572754 seconds ---
--- 9.86074709892273 seconds ---
--- 10.14663290977478 seconds ---
--- 12.202579259872437 seconds ---
--- 12.64861798286438 seconds ---
--- 7.173828840255737 seconds ---
--- 7.3388142585754395 seconds ---
--- 7.720240116119385 seconds ---
--- 8.277107000350952 seconds ---
--- 8.521019220352173 seconds ---
--- 8.828892230987549 seconds ---
--- 9.180794954299927 seconds ---
--- 8.264697790145874 seconds ---
--- 7.998165130615234 seconds ---
--- 8.391005277633667 seconds ---
--- 9.062461853027344 seconds ---
--- 8.369225025177002 seconds ---
--- 8.401145219802856 seconds ---
--- 9.3355131149292 seconds ---
--- 8.727741003036499 seconds ---
--- 8.797626972198486 seconds ---
--- 10.866436958312988 seconds ---
--- 9.888555765151978 seconds ---
--- 8.874553918838501 seconds ---
--- 8.376813173294067 seconds ---
--- 10.01473093032837 seconds ---
--- 12.42413592338562 seconds ---
--- 10.443741083145142 seconds ---
--- 9.162483930587769 seconds ---
--- 7.866487

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [8]:
# for file_name in nature_file_list+city_file_list:
#     print(file_name)
#     run_preproc(file_name)